# California Traffic Collision Data from SWITRS

Table `collisions` contains information about the collision, where it happened, what vehicles were involved.  

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import plotly.express as px
import folium
from folium import plugins
from folium.plugins import HeatMap
import pygal 
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.graph_objects as go
from pygal.style import Style
from IPython.display import display, HTML
plotly.offline.init_notebook_mode (connected = True)

## Import and View data

In [2]:
# Create a SQL connection to SQLite database
con = sqlite3.connect('/Users/sunhe/Desktop/NUS_Semester1/DSA5104/project/data/switrs.sqlite')

In [3]:
# check attributes in the table
collisions = pd.read_sql_query(
    """
    SELECT * 
    FROM collisions;
    """, con)

In [4]:
collisions.shape

(9424334, 75)

In [5]:
collisions.head()

,case_id,jurisdiction,officer_id,reporting_district,chp_shift,population,county_city_location,county_location,special_condition,beat_type,...,bicyclist_injured_count,motorcyclist_killed_count,motorcyclist_injured_count,primary_ramp,secondary_ramp,latitude,longitude,collision_date,collision_time,process_date
0,0081715,1941.0,11342,212,not chp,>250000,1941,los angeles,0,not chp,...,0,0,0.0,None,None,NaN,NaN,2020-03-14,07:45:00,2020-06-22
1,0726202,3600.0,8945,064,not chp,50000 to 100000,3612,san bernardino,0,not chp,...,0,0,0.0,None,None,NaN,NaN,2020-07-26,02:50:00,2020-09-30
2,3858022,1005.0,P379,2C,not chp,>250000,1005,fresno,0,not chp,...,0,0,0.0,None,None,NaN,NaN,2009-02-03,17:11:00,2009-04-27
3,3899441,9120.0,17248,None,2200 thru 0559,2500 to 10000,0801,del norte,0,chp state highway,...,0,0,0.0,None,None,NaN,NaN,2009-02-28,01:45:00,2009-11-02
4,3899442,9530.0,19005,None,0600 thru 1359,>250000,1942,los angeles,0,chp state highway,...,0,0,0.0,None,None,33.86465,-118.28533,2009-02-09,10:20:00,2010-01-14


## Analyze and visualize data

### How many accidents were recorded?

In [6]:
# Number of accidents recorded
df_total = pd.read_sql_query(
    """
    SELECT COUNT(case_id) AS Number_of_road_incidents
    FROM collisions;
    """, con)
df_total

,Number_of_road_incidents
0,9424334


### How has the number of collisions in California changed over the years?

In [7]:
# count of collisions per year since (year 2001 - 2021)
query = """
        SELECT count(*) as count_of_collisions, 
               strftime('%Y',collision_date) as year  
        FROM collisions
        GROUP BY year
        ORDER BY year
        
        """
table = pd.read_sql_query(query, con)

# plot
fig = px.line(table, x="year", y="count_of_collisions", title='Number of Collisions in California (2001 - 2021)')
fig.update_layout(
    xaxis_title="Year",
    yaxis_title="Number of collisions",
)
fig.show()

### What months have the highest car accidents in California?

In [8]:
# count of collisions based on month of the year (year 2001 - 2021)
query = """
        SELECT count(*) as count_of_collisions, 
               strftime('%m',collision_date) as month  
        FROM collisions
        GROUP BY month
        ORDER BY month
        
        """
table = pd.read_sql_query(query, con)

# plot 
fig = px.bar(table, x='month', y='count_of_collisions', title='Total Number of Collisions in California (2001 - 2021) based on month of year')
fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Collisions",
    xaxis = dict(
        tickmode = 'array',
        tickvals = [0,1,2,3,4,5,6,7,8,9,11,12],
        ticktext = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
    )
)

fig.show()

### Which are the top 10 counties in California with the highest collisions?

In [9]:
# Top 10 counties for collision counts in year 2020
query = """
        SELECT county_location, 
               count(county_location) as collisions_count
        FROM collisions
        WHERE strftime('%Y',collision_date) = '2020'
        GROUP BY county_location
        ORDER BY collisions_count desc
        LIMIT 10
        """

table = pd.read_sql_query(query, con)
table

,county_location,collisions_count
0,los angeles,111176
1,san bernardino,24322
2,riverside,22295
3,orange,21024
4,san diego,20168
5,alameda,16652
6,sacramento,16402
7,san joaquin,10326
8,kern,9861
9,santa clara,9860


### What was the percentage distribution of traffic accidents by severity?

In [10]:
# Declaration of colors and color palettes for visualization
plot_dictcolor = {'Property damage only': 'rgb(104,158,207)',
                  'Severe injury':        'rgb(245,87,91)',
                  'Other injury':         'rgb(245,162,87)',
                  'Fatal':                'rgb(161,161,161)',
                  'Pain':                 'rgb(245,214,87)'}

plot_dictcolor = {'Property damage only': 'rgb(104,158,207)',
                  'Severe injury':        'rgb(245,87,91)',
                  'Other injury':         'rgb(245,162,87)',
                  'Fatal':                'rgb(161,161,161)',
                  'Pain':                 'rgb(245,214,87)'}

plot_bgcolor = 'rgb(249,249,249)'
plot_linecolor = 'rgb(120,161,120)'

In [11]:
# What was the percentage distribution of traffic accidents by severity?
df_severity = pd.read_sql_query(
    """
    SELECT collision_severity AS Severity, 
           ROUND((COUNT(case_id) * 100.0) / (SELECT COUNT(case_id) FROM collisions), 1) AS Percentage
    FROM collisions
    WHERE collision_severity <> 'N'
    GROUP BY Severity
    ORDER BY COUNT(case_id) DESC;
    """, con)

df_severity['Severity'] = df_severity['Severity'].str.capitalize()

In [12]:
fig_severity = px.pie(
    df_severity, 
    values = 'Percentage', 
    names = 'Severity', 
    color = 'Severity',
    title = 'Completion of individual traffic incidents',
    color_discrete_map = plot_dictcolor,
    hole = 0.3)

fig_severity.show()

### How many traffic accidents were there in each year by severity?

In [13]:
# How many traffic accidents were there in each year by severity?
df_year_severity = pd.read_sql_query(
    """
    SELECT STRFTIME('%Y', collision_date) AS Year, 
           collision_severity AS Severity, COUNT(case_id) AS Number_of_road_incidents
    FROM collisions
    WHERE Severity <> 'N'
    GROUP BY Year, Severity;
    """, con)

df_year_severity['Severity'] = df_year_severity['Severity'].str.capitalize()

In [14]:
fig_year_severity = px.bar(
    df_year_severity, 
    x = 'Year', 
    y = 'Number_of_road_incidents', 
    color = 'Severity', 
    title = 'Number of traffic incidents by severity (by year)', 
    color_discrete_map = plot_dictcolor)

fig_year_severity.update_layout(plot_bgcolor = plot_bgcolor)
fig_year_severity.show()

### What were the three most common causes of traffic accidents each year individually?

In [15]:
# What were the three most common causes of traffic accidents each year individually?
df_reason = pd.read_sql_query(
"""
SELECT Year, Category, Percentage_of_annual_road_incidents 
FROM (SELECT *, DENSE_RANK() OVER (PARTITION BY Year ORDER BY Percentage_of_annual_road_incidents DESC) AS Rank
      FROM (SELECT Tab2.Year, 
                   Tab2.Category, 
                   ROUND((Tab2.Subtotal * 100.0) / (Tab1.Annual_total_of_accidents), 1) AS Percentage_of_annual_road_incidents
            FROM (SELECT STRFTIME('%Y', collision_date) AS Year, pcf_violation_category AS Category, COUNT(case_id) 
                                                                                                     AS Subtotal
                  FROM collisions
                  GROUP BY Year, Category) AS Tab2 
                  JOIN (SELECT STRFTIME('%Y', collision_date) AS Year, COUNT(case_id) AS Annual_total_of_accidents
                        FROM collisions
                        GROUP BY Year) AS Tab1 ON Tab1.Year = Tab2.Year 
            GROUP BY Tab2.Year, Tab2.Category, Percentage_of_annual_road_incidents))
WHERE Rank <= 3;
""", con)

df_reason['Category'] = df_reason['Category'].str.capitalize()
df_reason['Category'] = df_reason['Category'].str.replace('Dui', 'DUI')

In [19]:
fig_reason = px.line_polar(
    df_reason,
    r = 'Percentage_of_annual_road_incidents',
    theta = 'Year',
    color = 'Category',
    title = 'Percentage of accidents in years by dominant categories of road accident causes',
    color_discrete_sequence = px.colors.qualitative.Set2,
    markers = True)

fig_reason.update_layout(polar_bgcolor = plot_bgcolor)
fig_reason.show()

### What's the location of coliisions in Los angeles county in 2020?

In [17]:
# location of coliisions in Los angeles in 2020
query = """
        SELECT latitude, longitude 
        FROM collisions
        WHERE strftime('%Y',collision_date) = '2020'
        AND county_location = 'los angeles'
        AND latitude != ""
        AND longitude != ""

        """
table = pd.read_sql_query(query, con)

In [18]:
LA_COORDINATES = (34.0522, -118.2436)

# create empty map zoomed in on Los angeles
heat_map = folium.Map(location= LA_COORDINATES, zoom_start=10)
# heat map of collisions
HeatMap(table).add_to(folium.FeatureGroup(name='Heat Map').add_to(heat_map))
folium.LayerControl().add_to(heat_map)

display(heat_map)

## Clean data

In [20]:
# drop duplicate values
collisions = collisions.drop_duplicates() 

In [21]:
# check missing values
collisions.isnull().sum()

case_id                     0
jurisdiction            11407
officer_id              22367
reporting_district    5572350
chp_shift                   0
                       ...   
latitude              6730338
longitude             6730338
collision_date              0
collision_time          82415
process_date                0
Length: 75, dtype: int64

In [22]:
# How much percentage of values are missing from the total
missing = round((collisions.isnull().sum() * 100) / collisions.shape[0], 3)
missing_list_collisons = []
for i in range(0, len(missing)):
    missing_list_collisons.append(str(missing[i]) + ' %')
missing_collisons = pd.DataFrame(data = missing_list_collisons, index = missing.index, columns = ['missing percentage of collisions'])
missing_collisons

,missing percentage of collisions
case_id,0.0 %
jurisdiction,0.121 %
officer_id,0.237 %
reporting_district,59.127 %
chp_shift,0.0 %
...,...
latitude,71.414 %
longitude,71.414 %
collision_date,0.0 %
collision_time,0.874 %


In [23]:
# Drop missing values with a missing percentage of less than 10%
drop_index = []
for i in range(len(missing)):
    if missing[i] < 10:
        drop_index.append(missing.index[i])
drop_index

['case_id',
 'jurisdiction',
 'officer_id',
 'chp_shift',
 'population',
 'county_city_location',
 'county_location',
 'special_condition',
 'beat_type',
 'chp_beat_type',
 'chp_beat_class',
 'beat_number',
 'primary_road',
 'secondary_road',
 'distance',
 'intersection',
 'weather_1',
 'state_highway_indicator',
 'tow_away',
 'collision_severity',
 'killed_victims',
 'injured_victims',
 'party_count',
 'primary_collision_factor',
 'pcf_violation_category',
 'pcf_violation',
 'hit_and_run',
 'type_of_collision',
 'motor_vehicle_involved_with',
 'pedestrian_action',
 'road_surface',
 'road_condition_1',
 'lighting',
 'control_device',
 'chp_road_type',
 'pedestrian_collision',
 'bicycle_collision',
 'motorcycle_collision',
 'truck_collision',
 'not_private_property',
 'severe_injury_count',
 'other_visible_injury_count',
 'complaint_of_pain_injury_count',
 'pedestrian_killed_count',
 'pedestrian_injured_count',
 'bicyclist_killed_count',
 'bicyclist_injured_count',
 'motorcyclist_killed

In [25]:
new_collisons = collisions.dropna(subset=drop_index)

In [27]:
new_collisons.shape

(7230122, 75)

In [28]:
new_collisons.isnull().sum()

case_id                     0
jurisdiction                0
officer_id                  0
reporting_district    4627023
chp_shift                   0
                       ...   
latitude              4795648
longitude             4795648
collision_date              0
collision_time              0
process_date                0
Length: 75, dtype: int64

In [29]:
# Save cleaned data
with open('/Users/sunhe/Desktop/NUS_Semester1/DSA5104/project/data/clean_data/clean_collisions.csv',
          'a', encoding='utf8', newline="") as f:
    new_collisons.to_csv(f, header=True, index=0)

## Generate data